In [94]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [95]:
!setsid nohup ollama serve > run_serve.txt 2>&1 &

In [96]:
!ollama list

NAME    	ID          	SIZE  	MODIFIED          
gemma:2b	b50d6c999e59	1.7 GB	About an hour ago	


In [97]:
!setsid ollama run gemma:2b > run_gemma_2b.txt 2>&1 &

## LLM + Prompt

In [98]:
from langchain_community.llms import Ollama

llm = Ollama(model="gemma:2b")

llm.invoke("在2024年，特朗普在竞选集会现场遭到枪击，请问发生在哪个地点？")

'我无法提供关于特朗普在 2024 年竞选集会现场遭到枪击的信息。'

In [100]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一名新闻记者，很熟悉最近的重大新闻事件。"),
    ("user", "{input}")
])

chain = prompt | llm

chain.invoke({"input": "在2024年，特朗普在竞选集会现场遭到枪击，请问发生在哪个地点？"})

'我无法提供有关特定事件或人物的具体地点信息。'

In [101]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

chain.invoke({"input": "在2024年，特朗普在竞选集会现场遭到枪击，请问发生在哪个地点？"})

'我无法提供有关2024年美国总统竞选集会现场枪击的信息。我推荐您查询相关新闻或其他可靠的新闻来源。'

## RAG Retrieval Chain

In [102]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://www.thepaper.cn/newsDetail_forward_28067225")
docs = loader.load()

embeddings = OllamaEmbeddings(model="gemma:2b")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100, add_start_index=True)
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()

llm = Ollama(model="gemma:2b")

prompt = ChatPromptTemplate.from_template(
    """根据上下文的信息，回答指定问题:
    <context>
    {context}
    </context>
    Question: {input}
    """)

from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt)

from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke(
    {"input": "在2024年，特朗普在竞选集会现场遭到枪击，请问发生在哪个地点？"})

print(response["answer"])

2024年，特朗普在宾夕法尼亚州巴特勒市的竞选集会现场遭到枪击。


In [103]:
response = retrieval_chain.invoke(
    {"input": "拜登竞选团队在枪击事件后怎么做？"})

print(response["answer"])

根据上下文信息，拜登竞选团队在枪击事件后暂停所有外出活动，并尽快撤下所有的电视竞选广告。


## Conversation Retrieval Chain

In [104]:
from langchain_core.prompts import MessagesPlaceholder

from langchain_core.messages import HumanMessage, AIMessage


In [105]:
# Answer the user's questions

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("system", "根据上下文的信息，回答指定问题：\n\n{context}"),
])

llm = Ollama(model="gemma:2b")

document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever, document_chain)

chat_history = [
    HumanMessage(content="在2024年，特朗普在竞选集会现场遭到枪击，请问发生在哪个地点？"),
    AIMessage(content="2024年，特朗普在宾夕法尼亚州巴特勒市的竞选集会现场遭到枪击。")
]

response = retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "刺杀特朗普的凶手是谁？"
})

print(response["answer"])

2024年，特朗普在宾夕法尼亚州巴特勒市的竞选集会现场遭到枪击。
